In [1]:
import cv2
import numpy as np
import math
import operator
import pandas as pd
import numpy as np

In [2]:
def line_intersection(line1, line2):
    """
    Usage: let A = (x1, y1), B = (x2, y2), C = (x3, y3), D = (x4, y4)
            let l1 = (A, B), l2 = (C, D)
            call line_intersection(l1, l2)
            or directly call line_intersection((A, B), (C, D))
    Args: 
        line1 (2 points tuple): (A, B): A is first point coordinates (x11,y11), B is second point coordinates (x12,y12)
        line1 (2 points tuple): (C, D): C is first point coordinates (x21,y21), D is second point coordinates (x22,y22)
    Returns:
        is_intersecting (Boolean): True if intersection found, else False
        (x, y) (tuple of integers): coordinates of intersection point if found, else None
    """
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return False, (None, None)
    else:
        d = (det(*line1), det(*line2))
        x = det(d, xdiff) / div
        y = det(d, ydiff) / div
        return True, (x, y)

In [3]:
def detect_lines(img_in):
    img = img_in.copy()
#    cv2.imshow('img_in',img)
#    cv2.waitKey(0)

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#    cv2.imshow('gray', gray)
#    cv2.waitKey(0)
    edges = cv2.Canny(gray, 50, 200, 5)
#    cv2.imshow('edges', edges)
#    cv2.waitKey(0)
    """
    lines_p = cv2.HoughLinesP(edges, rho = 1, theta = 1*np.pi/180, threshold = 35 ,minLineLength = 10, maxLineGap = 25)
    print("Hough lines P: ")
    if lines_p is not None:
        for line in lines_p:
            print(line)
            x1, y1, x2, y2 = line[0]
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)

#    cv2.imshow('houghlines P',img)
#    cv2.waitKey(0)
    """
    
#    img2 = img_in.copy()
    lines = cv2.HoughLines(edges, rho = 1, theta = 1*np.pi/180, threshold = 20)
    extract_lines = []
    print("Hough lines: ")
    if lines is not None:
        for line in lines: 
            for rho,theta in line:
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
                theta_deg = round(np.rad2deg(theta))
#                cv2.line(img2,(x1,y1),(x2,y2),(0,0,255),2)
                ext_line = ((x1,y1), (x2,y2), theta_deg)
                print(line, ext_line)
                extract_lines.append(ext_line)
    
#    cv2.imshow('houghlines',img2)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()
    return extract_lines

In [4]:
red_bounds = ([0, 0, 60], [50, 50, 255])          # red
orange_bounds = ([0, 100, 200], [50, 200, 255])   # orange
yellow_bounds = ([0, 150, 100], [50, 255, 255])   # yellow
white_bounds = ([250, 250, 250], [255, 255, 255]) # white

In [5]:
######### TODO define this function
def get_valid_sign_squares(lines):
    """
    input: list of lines in the format (see the caller in stop_sign_detection)
    return: list of squares in the format [[(x1,y1), side1], [....]]
    """
    return [[(0,0),0]]

In [6]:
def stop_sign_detection(img_in):
    """Finds the centroid coordinates of a stop sign in the provided
    image.

    Args:
        img_in (numpy.array): image containing a traffic light.

    Returns:
        (x,y) tuple of the coordinates of the center of the stop sign.
    """
    blur = cv2.medianBlur(img_in,9)
#    cv2.imshow("blur", blur)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()
    color_boundaries = red_bounds
    lower = np.array(color_boundaries[0], dtype = "uint8")
    upper = np.array(color_boundaries[1], dtype = "uint8")
    mask = cv2.inRange(blur, lower, upper)
    """
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    #mask = cv2.dilate(mask,kernel,iterations = 1)
    cv2.imshow("mask", mask)
    cv2.waitKey(0)
    """
    out_img = cv2.bitwise_and(img_in, img_in, mask = mask)
#   out_img = cv2.dilate(out_img,kernel,iterations = 3)

    lines = detect_lines(out_img)
    allowed_line_angles = [45,315,135]
    allowed_lines = []
    if lines is not None:
        for line in lines:
            if line[2] in allowed_line_angles:
                cv2.line(out_img, line[0], line[1], (0,0,255),2) # TODO this is for lines demo only, comment
                allowed_lines.append(line)

        ##### TODO your code here #########
        squares = get_valid_sign_squares(allowed_lines)
        ## TODO show squares centers on out_img
        ## TODO comment out shwoing lines on out_img 6 lines above
    
    return out_img,allowed_lines

In [7]:
def warning_sign_detection(img_in):
    """Finds the centroid coordinates of a warning sign in the
    provided image.

    Args:
        img_in (numpy.array): image containing a traffic light.

    Returns:
        (x,y) tuple of the coordinates of the center of the sign.
    """
    
    blur = cv2.medianBlur(img_in,9)
#    cv2.imshow("blur", blur)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()
    color_boundaries = yellow_bounds
    lower = np.array(color_boundaries[0], dtype = "uint8")
    upper = np.array(color_boundaries[1], dtype = "uint8")
    mask = cv2.inRange(blur, lower, upper)

    out_img = cv2.bitwise_and(img_in, img_in, mask = mask)

    lines = detect_lines(out_img)
    allowed_line_angles = [45,315,135]
    allowed_lines = []
    if lines is not None:
        for line in lines:
            if line[2] in allowed_line_angles:
                cv2.line(out_img, line[0], line[1], (0,0,255),2) # TODO this is for lines demo only, comment
                allowed_lines.append(line)

        ##### TODO your code here #########
        squares = get_valid_sign_squares(allowed_lines)
        ## TODO show squares centers on out_img
        ## TODO comment out shwoing lines on out_img 6 lines above
    
    return out_img

In [8]:
def construction_sign_detection(img_in):
    """Finds the centroid coordinates of a construction sign in the
    provided image.

    Args:
        img_in (numpy.array): image containing a traffic light.

    Returns:
        (x,y) tuple of the coordinates of the center of the sign.
    """
    
    blur = cv2.medianBlur(img_in,9)
#    cv2.imshow("blur", blur)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()
    color_boundaries = orange_bounds
    lower = np.array(color_boundaries[0], dtype = "uint8")
    upper = np.array(color_boundaries[1], dtype = "uint8")
    mask = cv2.inRange(blur, lower, upper)

    out_img = cv2.bitwise_and(img_in, img_in, mask = mask)

    lines = detect_lines(out_img)
    allowed_line_angles = [45,315,135]
    allowed_lines = []
    if lines is not None:
        for line in lines:
            if line[2] in allowed_line_angles:
                cv2.line(out_img, line[0], line[1], (0,0,255),2) # TODO this is for lines demo only, comment
                allowed_lines.append(line)

        ##### TODO your code here #########
        squares = get_valid_sign_squares(allowed_lines)
        ## TODO show squares centers on out_img
        ## TODO comment out shwoing lines on out_img 6 lines above
    
    return out_img

In [9]:
# ############# MILESTONE 2 #############

# #input_images = ['scene_yld_1', 'scene_stp_1', 'scene_constr_1', 'scene_wrng_1', 'scene_dne_1', 
# #                'scene_some_signs', 'scene_all_signs', 'noisy_scene_all_signs', 'scene_all_signs_noisy']
# input_images = ['scene_some_signs', 'scene_stp_1', 'scene_all_signs', 'scene_all_signs_noisy']

# for img_in in input_images:
#     image = cv2.imread("input_images/{}.png".format(img_in))
#     cv2.imshow("image", image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
#     ret_img = stop_sign_detection(image)
#     cv2.imshow("detected STOP sign", ret_img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
#     ret_img = warning_sign_detection(image)
#     cv2.imshow("detected WARNING sign", ret_img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [42]:
input_images = ['scene_some_signs', 'scene_stp_1', 'scene_all_signs', 'scene_all_signs_noisy']
input_images=["scene_all_signs"]
for img_in in input_images:
    image = cv2.imread("input_images/{}.png".format(img_in))
    cv2.imshow("image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    ret_img,lines = stop_sign_detection(image)
    cv2.imshow("detected STOP sign", ret_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Hough lines: 
[[299.          1.5707964]] ((-1000, 298), (999, 299), 90)
[[308.          1.5707964]] ((-1000, 307), (999, 308), 90)
[[-239.           2.6179938]] ((-293, -985), (706, 746), 150)
[[317.         1.553343]] ((-994, 334), (1005, 299), 89)
[[339.          1.5707964]] ((-1000, 338), (999, 339), 90)
[[335.          1.5882496]] ((-1005, 317), (994, 352), 91)
[[6.400000e+02 5.235988e-01]] ((54, 1186), (1054, -546), 30)
[[-249.           2.6179938]] ((-284, -990), (715, 741), 150)
[[6.290000e+02 5.235988e-01]] ((44, 1180), (1044, -551), 30)
[[347.          1.5358897]] ((-987, 381), (1011, 311), 88)
[[343.         1.553343]] ((-993, 360), (1005, 325), 89)
[[-248.           2.6354473]] ((-267, -994), (701, 754), 151)
[[290.          1.5882496]] ((-1004, 272), (994, 307), 91)
[[317.          1.6231562]] ((-1015, 264), (982, 368), 93)
[[6.390000e+02 5.061455e-01]] ((74, 1184), (1043, -564), 29)
[[6.280000e+02 5.061455e-01]] ((64, 1179), (1034, -570), 29)
[[326.          1.6231562]] (

[[-379.           3.0892327]] ((326, -1018), (430, 978), 177)
[[-280.           3.0892327]] ((227, -1013), (331, 983), 177)
[[-331.           3.1241393]] ((313, -1005), (348, 994), 179)


In [43]:

def centroid(vertexes):
    x_list = [vertex [0] for vertex in vertexes]
    y_list = [vertex [1] for vertex in vertexes]
    l = len(vertexes)
    x = sum(x_list) / l
    y = sum(y_list) / l
    return(x,y)

def PolygonArea(corners):
    n = len(corners) # of corners
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area

def allmost_equal(x,y):
    if x-1.5 < y < x+1.5:
        return True
    return False


def duplicates(lst, item):
    return [i for i, x in enumerate(lst) if allmost_equal(x,item)]

In [44]:
def cal_center_side(lines):

    intersection_points=[]
    distances=[]
    all_lines_angle = []
    one_side_length = 0
    center = 0

    for i in range(len(lines)):
        for j in range(i+1):
            result=line_intersection(lines[i][:2],lines[j][:2])
            if result[0]:
                intersection_point = {}
                intersection_point["POINT"] = result[1]
                intersection_points.append(intersection_point)
    
    #finding all possible line angle which help to calculate distance or not
    for i in lines:
        all_lines_angle.append(i[2])
       
    if len(intersection_points)>=4:
        #finding distance of a point from all other who lines on its L 
        for i in range(len(intersection_points)):
            temp=[]
            for j in range(len(intersection_points)):
                x1,y1=intersection_points[i]["POINT"]
                x2,y2=intersection_points[j]["POINT"]
                if (x2-x1) == 0:
                    slope = np.infty
                else:
                    slope = (y2-y1) / (x2-x1)
                angle = math.degrees(math.atan(slope))
                if abs(angle) in all_lines_angle:
                    dist = round(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ),3)
                    temp.append(dist)
            intersection_points[i]["DISTANCES"] = temp
        
        #finding possible square
        possible_square = []
        for i in intersection_points:
        #     print(i)
            for j in i["DISTANCES"]:
        #         print(j)
        #         print(duplicates(i["DISTANCES"], j))
                if len(duplicates(i["DISTANCES"], j)) > 1:
        #             print(i["POINT"], "can be in square")
                    possible_square.append(i["POINT"])
                    break
                
        all_square =[]
        #extracting all sq
        for i in range(0,len(possible_square),4):
        #     print(possible_square[i])
            square = []
        #     if possible_square[i] and possible_square[i+1] and possible_square[i+2] and possible_square[i+3]:
            try:
                square.append(possible_square[i])
                square.append(possible_square[i+1])
                square.append(possible_square[i+2])
                square.append(possible_square[i+3])
            except:
                break
            all_square.append(square)
        all_area = []
        for i in all_square:
        #     print(i)
            all_area.append(PolygonArea(i))

        max_index, max_value = max(enumerate(all_area), key=operator.itemgetter(1))
        my_square = all_square[max_index]

        temp = ret_img
        for i in my_square:
#             print(i)
            temp_img = cv2.circle(temp, (int(i[0]),int(i[1])), 10, (255,0,0), -1)
        center = centroid(my_square)
#         print("THE CENTER IS",center)
        one_side_length = round(math.sqrt( (my_square[0][0] - my_square[1][0])**2 + (my_square[0][1] - my_square[1][1])**2 ),3)
#         print("one_side_length",one_side_length)
        cv2.imshow("detected STOP sign",temp_img )
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return (center,one_side_length)

In [45]:
print(cal_center_side(lines))


((348.5, 348.5), 97.581)


In [ ]:
possible_square

In [ ]:
# def find_point(distances,x,p1,p2):
#     print(p1,p2)
#     for i in range(0,len(distances)):
#         if i==p1: pass
#         else:
#             for j in range(0,len(distances[i])):
# #                 print("distance to find:",distances[i][j])

In [ ]:
# sqaure_sides = 4
# for i in range(0,len(distances)):
# #     print(distances[i])
#     for j in range(0,len(distances[i])):
# #         print("distance to find:",distances[i][j])
#         if i ==j:
#             pass
#         else:
#             find_point(distances,distances[i][j],i,j)

In [ ]:
# len(distances)